In [36]:
from PIL import Image

def overlay_mask_with_white(mask_path,original_path, out_path="overlaid_result.png", threshold=200):
    """
    For each pixel:
      - If mask pixel >= threshold → set output pixel = RED (255, 0, 0)
      - Else → keep original pixel
    """
    # Load original (RGB) and mask (grayscale)
    orig = Image.open(original_path).convert("RGB")
    mask = Image.open(mask_path).convert("L")

    # Resize mask if dimensions differ
    if mask.size != orig.size:
        mask = mask.resize(orig.size)

    # Get pixel access objects (fast for loops)
    orig_px = orig.load()
    mask_px = mask.load()

    # Create result image (same size/mode as original)
    result = Image.new("RGB", orig.size)
    result_px = result.load()

    # Iterate pixel by pixel
    width, height = orig.size
    for y in range(height):
        for x in range(width):
            if mask_px[x, y] >= threshold:
                result_px[x, y] = (255, 24, 45)  # RED
            else:
                result_px[x, y] = orig_px[x, y]  # keep original

    # Save result
    result.save(out_path)
    return out_path


# Example usage:
# overlay_mask_with_white("original.bmp", "mask.tiff", "output.png")


In [37]:
# ----------------- MAIN -----------------
root = "iris_db/IITD_v1"
segmented_root = "overlay"

orig_dir = os.path.join(root, "orig")
mask_dir = os.path.join(root, "mask_irisseg")

# Traverse subfolders in sorted order
for subfolder in sorted(os.listdir(orig_dir)):
    orig_sub = os.path.join(orig_dir, subfolder)
    mask_sub = os.path.join(mask_dir, subfolder)
    result_sub = os.path.join(segmented_root, subfolder)

    if not os.path.isdir(orig_sub):
        continue

    # Sorted files in each subfolder
    for filename in sorted(os.listdir(orig_sub)):
        if filename.lower().endswith(".bmp"):
            img_path = os.path.join(orig_sub, filename)

            # assume same stem but mask in .tiff
            stem = os.path.splitext(filename)[0]
            mask_path = os.path.join(mask_sub, stem + ".tiff")

            # force .png for saving
            save_path = os.path.join(result_sub, stem + ".bmp")

            if os.path.exists(mask_path):
                apply_mask(mask_path,img_path, save_path)
                print(f"Saved: {save_path}")
            else:
                print(f"Mask not found: {mask_path}")
            break
    break


Saved: overlay/100_L/100_L_1.bmp


In [38]:
from PIL import Image
import os

def overlay_mask_with_color(original_path, mask_path, out_path,
                            threshold=200,
                            overlay_color=(255, 0, 0),
                            alpha=0.5):
    """
    Overlay a mask onto an image with semi-transparent color.

    Args:
      original_path: path to original image (BMP, PNG, etc.)
      mask_path: path to mask image (TIFF, PNG, etc.)
      out_path: where to save result
      threshold: mask pixel intensity >= threshold is considered mask region
      overlay_color: RGB tuple used for overlay (e.g. (255,0,0) red)
      alpha: blending factor (0..1). 1=solid overlay, 0=none
    """
    # Load original (color) and mask (grayscale)
    orig = Image.open(original_path).convert("RGB")
    mask = Image.open(mask_path).convert("L")

    # Resize mask to original if needed
    if mask.size != orig.size:
        mask = mask.resize(orig.size, resample=Image.NEAREST)

    # Access pixel values
    orig_px = orig.load()
    mask_px = mask.load()
    width, height = orig.size

    # Create output image
    result = Image.new("RGB", orig.size)
    res_px = result.load()

    # Pre-split overlay color
    or_r, or_g, or_b = overlay_color

    # Pixel-by-pixel loop
    for y in range(height):
        for x in range(width):
            mval = mask_px[x, y]
            r0, g0, b0 = orig_px[x, y]

            if mval >= threshold:
                # Blend original pixel with overlay color
                r = int((1 - alpha) * r0 + alpha * or_r)
                g = int((1 - alpha) * g0 + alpha * or_g)
                b = int((1 - alpha) * b0 + alpha * or_b)
                res_px[x, y] = (r, g, b)
            else:
                # Keep original
                res_px[x, y] = (r0, g0, b0)

    # Ensure output folder exists
    os.makedirs(os.path.dirname(out_path) or ".", exist_ok=True)

    # Save result
    result.save(out_path)
    return result  # also return PIL image so we can show inside notebook


In [39]:
# Paths
original_path = "iris_db/IITD_v1/mask_irisseg/1_L/1_L_1.tiff"      # your BMP image
mask_path = "iris_db/IITD_v1/orig/1_L/1_L_1.bmp"         # your TIFF mask
out_path = "overlay_result.png" # save path

# Run overlay
res_img = overlay_mask_with_color(original_path, mask_path, out_path,
                                  threshold=200,
                                  overlay_color=(255,0,0), # red overlay
                                  alpha=0.5)

# Show inside notebook
res_img.show()
